In [ ]:
# =============================================================================
# STICK WITH EGGS GENERATOR
# Generates wooden sticks with realistic mosquito eggs and YOLO annotations
# =============================================================================

import os
import random
import numpy as np
from PIL import Image, ImageDraw, ImageEnhance, ImageFont, ImageFilter
from pathlib import Path
import json
import math

# =============================================================================
# CONFIGURATION
# =============================================================================

CONFIG = {
    # Canvas and stick dimensions
    'canvas_width': 640,
    'canvas_height': 1920,
    'actual_stick_width': 222,
    'actual_stick_height': 1766,
    'background_color': (0, 0, 0),
    
    # Background options
    'use_varied_backgrounds': True,
    'background_distribution': {
        'black': 0.3,
        'white_noise': 0.4,
        'plastic_texture': 0.3
    },
    'white_noise_density': 0.6,
    'white_noise_variation': 0.4,
    'plastic_texture_opacity_range': (0.45, 0.7),
    'plastic_texture_scale_range': (1.2, 2.5),
    'plastic_texture_position_variation': True,
    
    # Resource Paths (AI8 folder structure)
    'base_path': "D:/AI/datasets",
    'texture_base_path': "D:/AI/datasets/TEXTURES",
    'config_file_path': "D:/AI/datasets/TEXTURES/texture_selections_config2.json",
    'fonts_directory': "D:/AI/datasets/fonts",
    'plastic_texture_path': "D:/AI/datasets/TEXTURES/plastic_texture.png",
    'output_dir': 'D:/AI/datasets/D1.4',
    
    # Generation settings
    'num_sticks': 1500,
    'null_image_chance': 0.20,      # 20% chance of no eggs
    'save_metadata': True,
    'max_rotation_degrees': 12,
    'positioning_margin': 5,
    
    # Label settings
    'label_area_height_ratio': 1/6,
    'label_width_ratio': 1,
    'label_height_usage': 0.9,
    'label_y_position': 0,
    'font_size_multiplier': 0.33,
    'label_opacity': 0.9,
    'letter_offset_enabled': True,
    'letter_offset_range': 2,
    
    # Single letter effects
    'single_letter_effects': True,
    'single_letter_effect_chance': 0.7,
    'effect_letter_bold_size_multiplier': 1.5,
    
    # Ink wash effects
    'apply_ink_wash': True,
    'wash_opacity_reduction': 0.45,
    'wash_blur_radius': 1.8,
    'vertical_only_smudge': True,
    'intense_wash_drip_length': (60, 170),
    'intense_wash_opacity': (0.4, 1.0),
    'intense_wash_width': (4, 8),
    'intense_wash_num_drips': (8, 15),
    'apply_general_wash': True,
    'general_wash_chance': 0.2,
    'general_wash_length': (8, 25),
    'general_wash_intensity': (15, 40),
    
    # Scratch effects
    'apply_scratches': True,
    'scratch_count_range': (5, 30),
    'scratch_length_range': (50, 700),
    'scratch_width_range': (1, 10),
    'scratch_alpha_range': (80, 180),
    'scratches_over_labels': True,
    'parallel_scratch_chance': 0.5,
    'parallel_scratch_count': (2, 8),
    'parallel_scratch_spacing': (8, 20),
    
    # Overlay settings
    'no_overlay_chance': 0.25,      # 25% chance of no overlays
    'union_overlay_chance': 0.03,   # 3% chance for all overlays
    'overlays': {
        't221': {
            'chance': 0.05,
            'alpha_multiplier': (1.0, 4.0),
            'allow_y_shift': False,
            'allow_rotation': False,
            'rotation_range': (-45, 45),
            'allow_scaling': False,
            'scale_range': (0.5, 1.5),
            'allow_xy_movement': True,
            'x_movement_range': (0, 0),
            'y_movement_range': (-800, 800),
            'max_instances': 1,
            'color_shift': False
        },
        't222': {
            'chance': 0.05,
            'alpha_multiplier': (8.0, 15.0),
            'allow_y_shift': False,
            'allow_rotation': True,
            'rotation_range': (-45, 45),
            'allow_scaling': True,
            'scale_range': (0.5, 2.2),
            'allow_xy_movement': True,
            'x_movement_range': (-100, 100),
            'y_movement_range': (-800, 800),
            'max_instances': 2,
            'color_shift': True
        },
        't223': {
            'chance': 0.07,
            'alpha_multiplier': (3.5, 8.0),
            'allow_y_shift': False,
            'allow_rotation': True,
            'rotation_range': (-45, 45),
            'allow_scaling': True,
            'scale_range': (0.5, 3.3),
            'allow_xy_movement': True,
            'x_movement_range': (-120, 120),
            'y_movement_range': (-600, 600),
            'max_instances': 2,
            'color_shift': True
        }
    },
    
    # =============================================================================
    # EGG GENERATION SETTINGS
    # =============================================================================
    
    # Egg distribution (increased counts and more concentrated)
    'egg_distribution': {
        'minimal': {'range': (5, 15), 'mean': 10, 'std': 3, 'weight': 0.05},
        'typical': {'range': (20, 80), 'mean': 50, 'std': 15, 'weight': 0.45},
        'high': {'range': (100, 200), 'mean': 150, 'std': 25, 'weight': 0.45},
        'extreme': {'range': (250, 400), 'mean': 325, 'std': 40, 'weight': 0.05}
    },
    
    # Egg clustering (more concentrated around center)
    'cluster_probability': 0.75,
    'max_clusters_per_stick': 3,
    'eggs_per_cluster_min': 8,
    'eggs_per_cluster_max': 50,
    'cluster_radius': 100,
    'cluster_overlap_chance': 0.65,
    'edge_margin': 10,
    'placement_attempts': 200,
    
    # Center concentration settings
    'center_bias': 0.85,
    'center_area_ratio': 0.5,
    
    # Dataset split
    'train_ratio': 0.7,
    'val_ratio': 0.25,
    'test_ratio': 0.05,
}

# =============================================================================
# REALISTIC EGG GENERATOR
# =============================================================================

class RealisticEggGenerator:
    def __init__(self):
        # Real-world color ranges
        self.shade_ranges = {
            'S1': {'min': [114, 101, 79], 'max': [243, 204, 168]},   # Lightest
            'S2': {'min': [94, 67, 20], 'max': [154, 125, 91]},      # Medium-light
            'S3': {'min': [42, 29, 0], 'max': [118, 90, 56]},        # Medium-dark  
            'S4': {'min': [26, 7, 0], 'max': [80, 46, 25]}           # Darkest core
        }
        
        # Core pattern templates for S4 (darkest) pixels
        self.core_patterns = {
            'line_1x3': [(0, 0), (0, 1), (1, 0), (0, 2)],
            'line_2x3': [(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (1, 2)],
            'triangle_2x2_plus1': [(0, 0), (1, 0), (0, 1), (1, 1), (2, 1)],
            'diagonal_2x4': [(0, 0), (0, 1), (1, 2), (2, 2)]
        }
        
        # Size ranges (width x height in pixels)
        self.size_ranges = {
            'small': (3, 5),
            'medium': (3, 7), 
            'large': (4, 9),
            'max_diagonal': (5, 10)
        }
    
    def sample_color_from_shade(self, shade_level):
        """Sample a random color from the specified shade range"""
        range_data = self.shade_ranges[shade_level]
        return [
            random.randint(range_data['min'][0], range_data['max'][0]),
            random.randint(range_data['min'][1], range_data['max'][1]),
            random.randint(range_data['min'][2], range_data['max'][2])
        ]
    
    def blend_with_wood(self, egg_color, wood_color, intensity):
        """Blend egg color with wood color based on intensity"""
        return [
            int(wood_color[i] * (1 - intensity) + egg_color[i] * intensity)
            for i in range(3)
        ]
    
    def get_wood_color_at_position(self, image, x, y):
        """Sample wood color at specific position"""
        if 0 <= x < image.width and 0 <= y < image.height:
            return list(image.getpixel((x, y)))
        return [139, 119, 101]  # Default wood color
    
    def select_core_pattern(self, egg_width, egg_height):
        """Select appropriate core pattern based on egg size"""
        if egg_width <= 2 and egg_height <= 4:
            return random.choice(['line_1x3', 'diagonal_2x4'])
        elif egg_width <= 3:
            return random.choice(['line_1x3', 'line_2x3', 'diagonal_2x4'])
        else:
            return random.choice(['line_2x3', 'triangle_2x2_plus1'])
    
    def calculate_shade_level(self, distance_from_core, max_distance):
        """Calculate shade level based on distance from core"""
        if distance_from_core == 0:
            return 'S4'  # Core Shade
        
        normalized_distance = distance_from_core / max_distance
        
        if normalized_distance <= 0.3:
            return 'S3'
        elif normalized_distance <= 0.6:
            return 'S2'
        else:
            return 'S1'
    
    def create_realistic_egg(self, image, x, y, allow_overlap=False):
        """Create a realistic mosquito egg with proper fade and core patterns"""
        
        # Randomly select egg size
        size_type = random.choices(
            ['small', 'medium', 'large', 'max_diagonal'],
            weights=[0.2, 0.4, 0.25, 0.15]
        )[0]
        
        egg_width, egg_height = self.size_ranges[size_type]
        
        # Ensure minimum size
        egg_width = max(2, egg_width)
        egg_height = max(3, egg_height)
        
        # Check bounds
        if x + egg_width >= image.width or y + egg_height >= image.height:
            return None
        
        # Select core pattern
        core_pattern_name = self.select_core_pattern(egg_width, egg_height)
        core_pattern = self.core_patterns[core_pattern_name]
        
        # Sample colors for this egg
        egg_colors = {
            'S1': self.sample_color_from_shade('S1'),
            'S2': self.sample_color_from_shade('S2'), 
            'S3': self.sample_color_from_shade('S3'),
            'S4': self.sample_color_from_shade('S4')
        }
        
        # Create egg pixel by pixel
        egg_pixels = []
        max_distance = max(egg_width, egg_height) / 2
        
        # Calculate core center
        core_center_x = egg_width // 2
        core_center_y = egg_height // 2
        
        for rel_y in range(egg_height):
            for rel_x in range(egg_width):
                # Check if pixel is within egg shape (elliptical)
                dx = (rel_x - egg_width/2) / (egg_width/2)
                dy = (rel_y - egg_height/2) / (egg_height/2)
                
                if dx*dx + dy*dy <= 1.0:  # Inside ellipse
                    abs_x = x + rel_x
                    abs_y = y + rel_y
                    
                    # Get wood color at this position
                    wood_color = self.get_wood_color_at_position(image, abs_x, abs_y)
                    
                    # Check if this pixel is part of the core pattern
                    is_core = (rel_x - core_center_x, rel_y - core_center_y) in core_pattern
                    
                    if is_core:
                        shade_level = 'S4'
                        intensity = 0.9
                    else:
                        # Calculate distance from nearest core pixel
                        min_core_distance = min([
                            math.sqrt((rel_x - (core_center_x + cx))**2 + (rel_y - (core_center_y + cy))**2)
                            for cx, cy in core_pattern
                        ])
                        
                        shade_level = self.calculate_shade_level(min_core_distance, max_distance)
                        intensity = {
                            'S4': 0.9,
                            'S3': 0.75, 
                            'S2': 0.65,
                            'S1': 0.45
                        }[shade_level]
                    
                    # Blend egg color with wood
                    final_color = self.blend_with_wood(
                        egg_colors[shade_level], wood_color, intensity
                    )
                    
                    egg_pixels.append((abs_x, abs_y, tuple(final_color)))
        
        # Ensure image is RGBA
        if image.mode != 'RGBA':
            image = image.convert('RGBA')

        # Apply pixels to image
        image_array = np.array(image)
        for px, py, color in egg_pixels:
            if 0 <= px < image.width and 0 <= py < image.height:
                # Always use RGBA (add full alpha if needed)
                if len(color) == 3:
                    rgba_color = tuple(list(color) + [255])
                else:
                    rgba_color = color
                image_array[py, px] = rgba_color
                
        # Convert back to PIL image
        result_image = Image.fromarray(image_array)
        
        # Return YOLO annotation
        x_center = (x + egg_width / 2) / image.width
        y_center = (y + egg_height / 2) / image.height
        width = egg_width / image.width
        height = egg_height / image.height
        
        return result_image, f"0 {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}"

# =============================================================================
# COMBINED STICK AND EGG GENERATOR
# =============================================================================

def generate_organic_egg_count():
    """Generate egg count using weighted normal distributions"""
    if random.random() < 0.20:
        return 0
    
    distribution_config = CONFIG['egg_distribution']
    
    # Choose distribution category based on weights
    categories = list(distribution_config.keys())
    weights = [distribution_config[cat]['weight'] for cat in categories]
    
    chosen_category = np.random.choice(categories, p=weights)
    category_config = distribution_config[chosen_category]
    
    # Generate count using normal distribution
    mean = category_config['mean']
    std = category_config['std']
    min_val, max_val = category_config['range']
    
    # Sample from normal distribution and clamp to range
    egg_count = np.random.normal(mean, std)
    egg_count = np.clip(egg_count, min_val, max_val)
    egg_count = int(round(egg_count))
    
    return max(0, egg_count)

class CombinedStickEggGenerator:
    
    def __init__(self):
        self.setup_directories()
        self.load_fonts()
        self.load_texture_selections()
        self.load_background_resources()
        self.egg_generator = RealisticEggGenerator()
        
    def setup_directories(self):
        """Create output directory structure"""
        self.output_dir = Path(CONFIG['output_dir'])
        
        # Create YOLO dataset structure
        self.images_dir = self.output_dir / 'images'
        self.labels_dir = self.output_dir / 'labels'
        self.train_images_dir = self.images_dir / 'train'
        self.train_labels_dir = self.labels_dir / 'train'
        self.val_images_dir = self.images_dir / 'val'
        self.val_labels_dir = self.labels_dir / 'val'
        self.test_images_dir = self.images_dir / 'test'
        self.test_labels_dir = self.labels_dir / 'test'
        
        for dir_path in [self.train_images_dir, self.train_labels_dir, 
                        self.val_images_dir, self.val_labels_dir,
                        self.test_images_dir, self.test_labels_dir]:
            dir_path.mkdir(parents=True, exist_ok=True)
    
    def load_fonts(self):
        """Load available fonts"""
        self.regular_fonts = []
        self.bold_fonts = []
        
        if os.path.exists(CONFIG['fonts_directory']):
            for root, dirs, files in os.walk(CONFIG['fonts_directory']):
                for file in files:
                    if file.lower().endswith(('.ttf', '.otf')):
                        font_path = os.path.join(root, file)
                        if 'bold' in file.lower():
                            self.bold_fonts.append(font_path)
                        elif not any(variant in file.lower() for variant in ['italic', 'light', 'thin', 'medium']):
                            self.regular_fonts.append(font_path)
        
        self.fonts = self.regular_fonts if self.regular_fonts else self.bold_fonts
    
    def load_texture_selections(self):
        """Load texture selections from JSON file"""
        self.available_selections = []
        
        # Load from JSON config
        try:
            if os.path.exists(CONFIG['config_file_path']):
                with open(CONFIG['config_file_path'], 'r') as f:
                    texture_config = json.load(f)
                
                for texture_num_str, selections in texture_config.items():
                    texture_num = int(texture_num_str)
                    for sel_idx, selection_data in enumerate(selections):
                        self.available_selections.append({
                            'texture_num': texture_num,
                            'selection_index': sel_idx,
                            'selection_data': selection_data,
                            'source': 'json_config'
                        })
    
        except Exception as e:
            print(f" Error loading JSON config: {e}")
        
        # Add new wood textures
        new_wood_textures = {
            19: 1, 20: 1, 21: 1, 23: 3
        }
        for texture_num in new_wood_textures:
            texture_path = os.path.join(CONFIG['texture_base_path'], f"t{texture_num}.png")
            if os.path.exists(texture_path):
                self.available_selections.append({
                    'texture_num': texture_num,
                    'selection_index': 0,
                    'selection_data': {
                        'filename': f"t{texture_num}.png",
                        'x': 0, 'y': 0,
                        'crop_w': 1000, 'crop_h': 1000,
                        'scale': 1.0,
                        'rotation': 0,
                        'brightness': 1.0,
                        'contrast': 1.0,
                        'saturation': 1.0
                    },
                    'source': 'new_wood'
                })
        
        # Check for overlay textures
        self.overlay_textures = {}
        overlay_names = ['t221', 't222', 't223']
        
        for overlay_name in overlay_names:
            overlay_path = os.path.join(CONFIG['texture_base_path'], f"{overlay_name}.png")
            if os.path.exists(overlay_path):
                self.overlay_textures[overlay_name] = overlay_path
                settings = CONFIG['overlays'][overlay_name]
        
        if not self.available_selections:
            print(f" No textures found, using fallback")
    
    def load_background_resources(self):
        """Load plastic texture for background variants"""
        self.plastic_texture = None
        
        if CONFIG['use_varied_backgrounds']:
            plastic_path = CONFIG['plastic_texture_path']
            if os.path.exists(plastic_path):
                try:
                    self.plastic_texture = Image.open(plastic_path).convert("RGB")
                except Exception as e:
                    print(f" Error loading plastic texture: {e}")
                    self.plastic_texture = None
    
    # =============================================================================
    # LABEL GENERATION 
    # =============================================================================
    
    def generate_random_label_format(self):
        """Generate one of the 5 label formats"""
        format_choice = random.choices(
            ['format1', 'format2', 'format3', 'format4', 'format5'],
            weights=[20, 20, 20, 20, 20]
        )[0]
        
        if format_choice == 'format1':
            return self.generate_format1()
        elif format_choice == 'format2':
            return self.generate_format2()
        elif format_choice == 'format3':
            return self.generate_format3()
        elif format_choice == 'format4':
            return self.generate_format4()
        else:
            return self.generate_format5()
    
    def generate_format1(self):
        """Format 1: A-Z (red), NumberLetter, 0-30, date - 4 lines"""
        letter = random.choice('ABCDEFGHIJKLMNOPQRSTUVWXYZ')
        number = random.randint(10, 99)
        letters = random.choice(['HU', 'AB', 'CD', 'EF', 'GH', 'IJ', 'KL', 'MN', 'OP', 'QR', 'ST', 'UV', 'WX', 'YZ'])
        line2 = f"{number}{letters}"
        line3 = str(random.randint(0, 30))
        month, day, year = random.randint(1, 12), random.randint(1, 28), random.randint(24, 28)
        line4 = f"{month}/{day}/{year}"
        
        return {
            'format': 'format1',
            'lines': [
                {'text': letter, 'color': (200, 0, 0), 'opacity': 1.0},
                {'text': line2, 'color': random.choice([(0, 0, 0), (0, 0, 150)]), 'opacity': 0.9},
                {'text': line3, 'color': random.choice([(0, 0, 0), (0, 0, 150)]), 'opacity': 0.8},
                {'text': line4, 'color': random.choice([(0, 0, 0), (0, 0, 150)]), 'opacity': 0.7}
            ]
        }
    
    def generate_format2(self):
        """Format 2: A-Z (red), NumberLetter, date - 3 lines"""
        letter = random.choice('ABCDEFGHIJKLMNOPQRSTUVWXYZ')
        number = random.randint(10, 99)
        letters = random.choice(['HU', 'AB', 'CD', 'EF', 'GH', 'IJ', 'KL', 'MN', 'OP', 'QR', 'ST', 'UV', 'WX', 'YZ'])
        line2 = f"{number}{letters}"
        month, day, year = random.randint(1, 12), random.randint(1, 28), random.randint(2024, 2028)
        line3 = f"{month}/{day}/{str(year)[2:]}"
        
        return {
            'format': 'format2',
            'lines': [
                {'text': letter, 'color': (200, 0, 0), 'opacity': 1.0},
                {'text': line2, 'color': random.choice([(0, 0, 0), (0, 0, 150)]), 'opacity': 0.9},
                {'text': line3, 'color': random.choice([(0, 0, 0), (0, 0, 150)]), 'opacity': 0.8}
            ]
        }
    
    def generate_format3(self):
        """Format 3: DD.MM, year, F1-F9, optional LABASM - dark blue/black"""
        day, month = random.randint(1, 31), random.randint(1, 12)
        line1 = f"{day:02d}.{month:02d}"
        year = random.choice([2024, 2025, 2026, 2027])
        line2 = str(year)
        f_number = random.randint(1, 9)
        line3 = f"F{f_number}"
        
        lines = [
            {'text': line1, 'color': random.choice([(0, 0, 100), (0, 0, 0)]), 'opacity': 1.0},
            {'text': line2, 'color': random.choice([(0, 0, 100), (0, 0, 0)]), 'opacity': 0.9},
            {'text': line3, 'color': random.choice([(0, 0, 100), (0, 0, 0)]), 'opacity': 0.8}
        ]
        
        if random.random() < 0.05:
            small_words = ['LABASM', 'CTRL', 'TEST', 'SPEC', 'DATA']
            line4 = random.choice(small_words)
            lines.append({'text': line4, 'color': random.choice([(0, 0, 100), (0, 0, 0)]), 'opacity': 0.7})
        
        return {'format': 'format3', 'lines': lines}
    
    def generate_format4(self):
        """Format 4: BOT, 01-09, 2025, date - 4 lines"""
        bot_number = random.randint(1, 9)
        month, day = random.randint(1, 12), random.randint(1, 28)
        color = random.choice([(0, 0, 0), (0, 0, 150)])
        
        return {
            'format': 'format4',
            'lines': [
                {'text': 'BOT', 'color': color, 'opacity': 1.0},
                {'text': f"{bot_number:02d}", 'color': color, 'opacity': 0.9},
                {'text': '2025', 'color': color, 'opacity': 0.8},
                {'text': f"{month:02d}.{day:02d}", 'color': color, 'opacity': 0.7}
            ]
        }
    
    def generate_format5(self):
        """Format 5: Code, A/M alternating with date - dark blue gradient"""
        code_formats = [
            f"FRS{random.randint(1, 9)}", f"VL{random.randint(1, 20)}", f"PHP{random.randint(1, 9)}",
            f"JS{random.randint(1, 15)}", f"PY{random.randint(1, 12)}", "API", "SQL", "XML", "CMD"
        ]
        line1 = random.choice(code_formats)
        am_choice = random.choice(['A', 'M'])
        
        date_formats = [
            f"{random.randint(1, 31):02d}.{random.randint(1, 12):02d}.25",
            f"{random.randint(1, 12)}/{random.randint(1, 28)}/25",
            f"{random.randint(1, 31)}.{random.randint(1, 12)}.24"
        ]
        date_text = random.choice(date_formats)
        
        if random.choice([True, False]):
            line2, line3 = am_choice, date_text
        else:
            line2, line3 = date_text, am_choice
        
        return {
            'format': 'format5',
            'lines': [
                {'text': line1, 'color': (0, 0, 120), 'opacity': 1.0},
                {'text': line2, 'color': (0, 0, 100), 'opacity': 0.8},
                {'text': line3, 'color': (0, 0, 80), 'opacity': 0.6}
            ]
        }
    
    def create_multi_line_label(self, label_data, target_size, font_path):
        """Create multi-line label image"""
        img = Image.new('RGBA', target_size, (0, 0, 0, 0))
        draw = ImageDraw.Draw(img)
        
        lines = label_data['lines']
        num_lines = len(lines)
        
        # Calculate font size
        base_font_size = int(target_size[1] * CONFIG['font_size_multiplier'])
        if num_lines == 4:
            font_size = int(base_font_size * 0.8)
        elif num_lines == 3:
            font_size = int(base_font_size * 0.9)
        else:
            font_size = base_font_size
        
        # Load regular font
        try:
            regular_font = ImageFont.truetype(font_path, font_size)
        except:
            regular_font = ImageFont.load_default()
        
        # Load bold font for special letter
        try:
            if self.bold_fonts:
                bold_font_path = random.choice(self.bold_fonts)
                bold_font_size = int(font_size * CONFIG['effect_letter_bold_size_multiplier'])
                bold_font = ImageFont.truetype(bold_font_path, bold_font_size)
            else:
                bold_font_size = int(font_size * CONFIG['effect_letter_bold_size_multiplier'])
                bold_font = ImageFont.truetype(font_path, bold_font_size)
        except:
            bold_font = regular_font
        
        # Select the special effect letter
        effect_letter = self.select_effect_letter(label_data)
        
        # Calculate line spacing
        line_height = target_size[1] // (num_lines + 1)
        
        # Draw each line
        for line_idx, line_data in enumerate(lines):
            text = line_data['text']
            color = line_data['color']
            opacity = line_data.get('opacity', 1.0)
            
            # Position
            y = (line_idx + 1) * line_height - font_size // 2
            bbox = draw.textbbox((0, 0), text, font=regular_font)
            text_width = bbox[2] - bbox[0]
            x = (target_size[0] - text_width) // 2
            
            # Draw each character individually
            current_x = x
            for char_idx, char in enumerate(text):
                # Check if this is the special effect letter
                is_effect_letter = (effect_letter is not None and 
                                  effect_letter[0] == line_idx and 
                                  effect_letter[1] == char_idx)
                
                # Choose font and position
                if is_effect_letter:
                    use_font = bold_font
                    alpha = int(255 * opacity)
                    char_y = y - 5  # Move up slightly
                else:
                    use_font = regular_font
                    alpha = int(255 * opacity)
                    char_y = y
                    
                    # Apply letter offset
                    if CONFIG['letter_offset_enabled'] and len(text) > 1 and char_idx % 2 == 1:
                        offset_range = CONFIG['letter_offset_range']
                        current_x += random.randint(-offset_range, offset_range)
                        char_y += random.randint(-offset_range, offset_range)
                
                # Draw the character
                draw.text((current_x, char_y), char, font=use_font, fill=(*color, alpha))
                
                # Move to next character position
                char_bbox = draw.textbbox((0, 0), char, font=use_font)
                current_x += char_bbox[2] - char_bbox[0]
        
        # Apply overall opacity
        if CONFIG['label_opacity'] < 1.0:
            img = self.apply_opacity(img, CONFIG['label_opacity'])
        
        return img
    
    def select_effect_letter(self, label_data):
        """Select one random letter from all lines to apply special effects"""
        if not CONFIG['single_letter_effects'] or random.random() > CONFIG['single_letter_effect_chance']:
            return None
        
        # Count total letters across all lines
        letter_positions = []
        
        for line_idx, line_data in enumerate(label_data['lines']):
            text = line_data['text']
            for char_idx, char in enumerate(text):
                if char.isalnum():
                    letter_positions.append((line_idx, char_idx, char))
        
        if not letter_positions:
            return None
        
        # Select random letter
        selected_idx = random.randint(0, len(letter_positions) - 1)
        return letter_positions[selected_idx]
    
    def apply_opacity(self, img, opacity):
        """Apply overall opacity to image"""
        if img.mode == 'RGBA':
            r, g, b, a = img.split()
            alpha_pixels = a.load()
            width, height = a.size
            
            for y in range(height):
                for x in range(width):
                    if alpha_pixels[x, y] > 0:
                        alpha_pixels[x, y] = int(alpha_pixels[x, y] * opacity)
            
            return Image.merge('RGBA', (r, g, b, a))
        return img
    
    # =============================================================================
    # TEXTURE AND BACKGROUND CREATION
    # =============================================================================
    
    def load_texture(self, texture_selection):
        """Load texture from selection data"""
        try:
            filename = texture_selection['selection_data'].get('filename', f"t{texture_selection['texture_num']}.png")
            texture_path = os.path.join(CONFIG['texture_base_path'], filename)
            
            if not os.path.exists(texture_path):
                return None
            
            texture = Image.open(texture_path).convert("RGBA")
            
            # Handle new wood textures (direct use)
            if texture_selection['source'] == 'new_wood':
                texture = texture.resize((CONFIG['actual_stick_width'], CONFIG['actual_stick_height']), Image.Resampling.LANCZOS)
                return texture
            
            # Handle JSON config textures with cropping and processing
            selection_data = texture_selection['selection_data']
            
            # Apply rotation if specified
            rotation = selection_data.get('rotation', 0)
            if rotation != 0:
                if rotation == 90:
                    texture = texture.rotate(-90, expand=True)
                elif rotation == 180:
                    texture = texture.rotate(180, expand=True)
                elif rotation == 270:
                    texture = texture.rotate(90, expand=True)
            
            # Extract and apply selection parameters
            sel_x = selection_data.get('x', texture.size[0] // 2)
            sel_y = selection_data.get('y', texture.size[1] // 2)
            crop_w = selection_data.get('crop_w', 300)
            crop_h = selection_data.get('crop_h', 400)
            scale_factor = selection_data.get('scale', 1.0)
            
            # Calculate crop area with scaling
            scaled_crop_w = int(crop_w * scale_factor)
            scaled_crop_h = int(crop_h * scale_factor)
            half_w = scaled_crop_w // 2
            half_h = scaled_crop_h // 2
            
            crop_x1 = max(0, sel_x - half_w)
            crop_y1 = max(0, sel_y - half_h)
            crop_x2 = min(texture.size[0], sel_x + half_w)
            crop_y2 = min(texture.size[1], sel_y + half_h)
            
            # Validate crop area
            if crop_x2 - crop_x1 < 50 or crop_y2 - crop_y1 < 50:
                center_x, center_y = texture.size[0] // 2, texture.size[1] // 2
                crop_x1 = max(0, center_x - half_w)
                crop_y1 = max(0, center_y - half_h)
                crop_x2 = min(texture.size[0], center_x + half_w)
                crop_y2 = min(texture.size[1], center_y + half_h)
            
            # Crop and resize
            cropped_texture = texture.crop((crop_x1, crop_y1, crop_x2, crop_y2))
            final_texture = cropped_texture.resize((CONFIG['actual_stick_width'], CONFIG['actual_stick_height']), Image.Resampling.LANCZOS)
            
            # Apply adjustments
            adjustments = [
                ('brightness', ImageEnhance.Brightness),
                ('contrast', ImageEnhance.Contrast),
                ('saturation', ImageEnhance.Color)
            ]
            
            for adj_name, enhancer_class in adjustments:
                adj_value = selection_data.get(adj_name, 1.0)
                if adj_value != 1.0:
                    final_texture = enhancer_class(final_texture).enhance(adj_value)
            
            return final_texture
            
        except Exception as e:
            print(f" Error loading texture: {e}")
            return None
    
    def create_stick_mask(self):
        """Create stick shape mask"""
        mask = Image.new("L", (CONFIG['actual_stick_width'], CONFIG['actual_stick_height']), 0)
        draw = ImageDraw.Draw(mask)
        arc_height = CONFIG['actual_stick_width']
        
        draw.rectangle([0, arc_height // 2, CONFIG['actual_stick_width'], 
                       CONFIG['actual_stick_height'] - arc_height // 2], fill=255)
        draw.ellipse([0, 0, CONFIG['actual_stick_width'], arc_height], fill=255)
        draw.ellipse([0, CONFIG['actual_stick_height'] - arc_height, 
                     CONFIG['actual_stick_width'], CONFIG['actual_stick_height']], fill=255)
        return mask
    
    def create_base_stick(self, stick_index):
        """Create base stick with texture (overlays applied later)"""
        
        if self.available_selections:
            # Select texture
            non_wood_selections = [s for s in self.available_selections if s['texture_num'] > 0]
            available_textures = non_wood_selections if non_wood_selections else self.available_selections
            
            texture_index = stick_index % len(available_textures)
            texture_selection = available_textures[texture_index]
            
            texture = self.load_texture(texture_selection)
            if texture is None:
                texture = Image.new("RGBA", (CONFIG['actual_stick_width'], CONFIG['actual_stick_height']), (139, 119, 101, 255))
                texture_selection = {'texture_num': 0, 'source': 'fallback'}
        else:
            # Simple brown background
            texture = Image.new("RGBA", (CONFIG['actual_stick_width'], CONFIG['actual_stick_height']), (139, 119, 101, 255))
            texture_selection = {'texture_num': 0, 'source': 'fallback'}
        
        # Create stick with mask
        stick = Image.new("RGBA", (CONFIG['actual_stick_width'], CONFIG['actual_stick_height']), (0, 0, 0, 0))
        mask = self.create_stick_mask()
        stick.paste(texture, (0, 0), mask)
        
        return stick, texture_selection
    
    def apply_color_shift(self, overlay):
        """Shift overlay colors from black to brown/dark shades"""
        r, g, b, a = overlay.split()
        
        # Load pixel data
        r_pixels = r.load()
        g_pixels = g.load()
        b_pixels = b.load()
        width, height = overlay.size
        
        # Generate random brown shift values for this overlay instance
        brown_shift_r = random.randint(10, 40)  # Add red
        brown_shift_g = random.randint(5, 25)   # Add some green
        brown_shift_b = random.randint(0, 15)   # Add a little blue
        
        for y in range(height):
            for x in range(width):
                # Get original values
                orig_r = r_pixels[x, y]
                orig_g = g_pixels[x, y]
                orig_b = b_pixels[x, y]
                
                # Only shift darker colors (avoid changing light areas)
                if orig_r + orig_g + orig_b < 200:  # Dark pixel threshold
                    # Apply brown shift
                    new_r = min(255, orig_r + brown_shift_r)
                    new_g = min(255, orig_g + brown_shift_g)
                    new_b = min(255, orig_b + brown_shift_b)
                    
                    r_pixels[x, y] = new_r
                    g_pixels[x, y] = new_g
                    b_pixels[x, y] = new_b
        
        return Image.merge('RGBA', (r, g, b, a))
    
    def load_overlay_texture(self, overlay_name):
        """Load and transform overlay texture based on its settings"""
        try:
            if overlay_name not in self.overlay_textures:
                return None
            
            overlay_path = self.overlay_textures[overlay_name]
            settings = CONFIG['overlays'][overlay_name]
            
            overlay = Image.open(overlay_path).convert("RGBA")
            
            # Apply color shift if enabled
            if settings.get('color_shift', False):
                overlay = self.apply_color_shift(overlay)
            
            # Determine target size based on overlay type
            if settings.get('allow_scaling', False):
                # For scalable overlays, keep original size and then scale
                scale_factor = random.uniform(*settings['scale_range'])
                target_width = int(overlay.size[0] * scale_factor)
                target_height = int(overlay.size[1] * scale_factor)
            else:
                # For non-scalable overlays, resize to fit stick
                target_width = CONFIG['actual_stick_width']
                target_height = CONFIG['actual_stick_height']
            
            # Resize overlay
            overlay = overlay.resize((target_width, target_height), Image.Resampling.LANCZOS)
            
            # Apply rotation if allowed
            if settings.get('allow_rotation', False):
                rotation_angle = random.uniform(*settings['rotation_range'])
                overlay = overlay.rotate(rotation_angle, expand=True, fillcolor=(0, 0, 0, 0))
            
            # Create final positioned overlay
            final_overlay = Image.new('RGBA', (CONFIG['actual_stick_width'], CONFIG['actual_stick_height']), (0, 0, 0, 0))
            
            # Calculate position
            x_offset = (CONFIG['actual_stick_width'] - overlay.size[0]) // 2
            y_offset = (CONFIG['actual_stick_height'] - overlay.size[1]) // 2
            
            # Apply Y shift if allowed
            if settings.get('allow_y_shift', False):
                y_shift = random.randint(*settings['y_shift_range'])
                y_offset += y_shift
            
            # Apply XY movement if allowed
            if settings.get('allow_xy_movement', False):
                # Use separate X and Y ranges
                if 'x_movement_range' in settings and 'y_movement_range' in settings:
                    x_movement = random.randint(*settings['x_movement_range'])
                    y_movement = random.randint(*settings['y_movement_range'])
                else:
                    # Fallback to combined range (legacy)
                    x_movement = random.randint(*settings['xy_movement_range'])
                    y_movement = random.randint(*settings['xy_movement_range'])
                
                x_offset += x_movement
                y_offset += y_movement
            
            # Ensure overlay stays within bounds (allow partial off-screen for variety)
            x_offset = max(-overlay.size[0]//2, min(x_offset, CONFIG['actual_stick_width'] - overlay.size[0]//2))
            y_offset = max(-overlay.size[1]//2, min(y_offset, CONFIG['actual_stick_height'] - overlay.size[1]//2))
            
            # Paste overlay onto final canvas
            final_overlay.paste(overlay, (x_offset, y_offset), overlay)
            
            return final_overlay
            
        except Exception as e:
            print(f" Error loading overlay texture {overlay_name}: {e}")
            return None
    
    def apply_overlay_texture(self, base_texture, stick_index=0):
        """Apply overlay textures with union boost"""
        
        # 25% chance of NO overlays at all
        if random.random() < CONFIG['no_overlay_chance']:
            return base_texture, False, "none"
        
        # Simple union boost - force ALL overlays to appear
        union_chance = random.random()
        force_all_overlays = union_chance <= CONFIG['union_overlay_chance']
        
        applied_overlays = []
        result_texture = base_texture
        
        # Try each overlay type
        for overlay_name in ['t221', 't222', 't223']:
            if overlay_name not in self.overlay_textures:
                continue
            
            settings = CONFIG['overlays'][overlay_name]
            
            # Check if we should apply this overlay
            should_apply = False
            if force_all_overlays:
                should_apply = True
            else:
                overlay_chance = random.random()
                if overlay_chance <= settings['chance']:
                    should_apply = True
            
            if not should_apply:
                continue
            
            # Determine how many instances to apply
            max_instances = settings.get('max_instances', 1)
            num_instances = random.randint(1, max_instances)
            
            # Apply multiple instances
            for instance in range(num_instances):
                overlay = self.load_overlay_texture(overlay_name)
                if overlay is None:
                    continue
                
                try:
                    # Ensure both images are RGBA
                    if result_texture.mode != 'RGBA':
                        result_texture = result_texture.convert('RGBA')
                    if overlay.mode != 'RGBA':
                        overlay = overlay.convert('RGBA')
                    
                    # Enhance alpha based on settings
                    alpha_multiplier = settings['alpha_multiplier']
                    if isinstance(alpha_multiplier, tuple):
                        alpha_multiplier = random.uniform(*alpha_multiplier)
                    
                    r, g, b, a = overlay.split()
                    
                    # Apply alpha multiplier carefully
                    if alpha_multiplier > 1.0:
                        enhanced_alpha = a.point(lambda x: min(255, int(x * alpha_multiplier)) if x > 0 else 0)
                    else:
                        enhanced_alpha = a.point(lambda x: int(x * alpha_multiplier) if x > 0 else 0)
                    
                    overlay = Image.merge('RGBA', (r, g, b, enhanced_alpha))
                    
                    result_texture = Image.alpha_composite(result_texture, overlay)
                    
                    if instance == 0:  # Only add to list once per overlay type
                        applied_overlays.append(f"{overlay_name}x{num_instances}")
                    
                except Exception as e:
                    print(f" Error applying {overlay_name}: {e}")
        
        has_overlay = len(applied_overlays) > 0
        overlay_info = ", ".join(applied_overlays) if applied_overlays else "none"
        
        # Mark union mode in the info
        if force_all_overlays and has_overlay:
            overlay_info = f"UNION({overlay_info})"
        
        return result_texture, has_overlay, overlay_info
    
    def create_white_noise_background(self):
        """Create white noise background"""
        background = Image.new("RGB", (CONFIG['canvas_width'], CONFIG['canvas_height']), (0, 0, 0))
        pixels = background.load()
        base_density = CONFIG['white_noise_density']
        variation = CONFIG['white_noise_variation']
        
        density_variation = random.uniform(-variation, variation)
        actual_density = max(0.1, min(0.9, base_density + density_variation))
        
        for y in range(CONFIG['canvas_height']):
            for x in range(CONFIG['canvas_width']):
                if random.random() < actual_density:
                    if random.random() < 0.9:
                        gray_val = random.randint(5, 120)
                        pixels[x, y] = (gray_val, gray_val, gray_val)
                    else:
                        gray_val = random.randint(120, 160)
                        pixels[x, y] = (gray_val, gray_val, gray_val)
        
        return background
    
    def create_plastic_texture_background(self):
        """Create background using plastic texture"""
        if self.plastic_texture is None:
            return self.create_white_noise_background()
        
        black_bg = Image.new("RGB", (CONFIG['canvas_width'], CONFIG['canvas_height']), (0, 0, 0))
        
        # Scale and position plastic texture
        orig_width, orig_height = self.plastic_texture.size
        min_scale_x = CONFIG['canvas_width'] / orig_width
        min_scale_y = CONFIG['canvas_height'] / orig_height
        min_scale = max(min_scale_x, min_scale_y)
        
        additional_scale = random.uniform(*CONFIG['plastic_texture_scale_range'])
        final_scale = min_scale * additional_scale
        
        scaled_width = int(orig_width * final_scale)
        scaled_height = int(orig_height * final_scale)
        
        plastic_scaled = self.plastic_texture.resize((scaled_width, scaled_height), Image.Resampling.LANCZOS)
        
        # Position with variation
        if CONFIG['plastic_texture_position_variation']:
            max_x_offset = max(0, scaled_width - CONFIG['canvas_width'])
            max_y_offset = max(0, scaled_height - CONFIG['canvas_height'])
            x_offset = -random.randint(0, max_x_offset) if max_x_offset > 0 else 0
            y_offset = -random.randint(0, max_y_offset) if max_y_offset > 0 else 0
        else:
            x_offset = -(scaled_width - CONFIG['canvas_width']) // 2
            y_offset = -(scaled_height - CONFIG['canvas_height']) // 2
        
        # Apply opacity and composite
        plastic_rgba = plastic_scaled.convert("RGBA")
        black_bg_rgba = black_bg.convert("RGBA")
        
        opacity = random.uniform(*CONFIG['plastic_texture_opacity_range'])
        r, g, b, a = plastic_rgba.split()
        alpha_channel = Image.new('L', plastic_rgba.size, int(255 * opacity))
        plastic_with_alpha = Image.merge('RGBA', (r, g, b, alpha_channel))
        
        temp_canvas = Image.new("RGBA", (scaled_width, scaled_height), (0, 0, 0, 0))
        temp_canvas.paste(plastic_with_alpha, (0, 0), plastic_with_alpha)
        
        crop_x = -x_offset
        crop_y = -y_offset
        final_texture = temp_canvas.crop((
            crop_x, crop_y,
            crop_x + CONFIG['canvas_width'],
            crop_y + CONFIG['canvas_height']
        ))
        
        result = Image.alpha_composite(black_bg_rgba, final_texture)
        return result.convert("RGB")
    
    def create_background(self):
        """Create background based on configured distribution"""
        if not CONFIG['use_varied_backgrounds']:
            return Image.new("RGB", (CONFIG['canvas_width'], CONFIG['canvas_height']), CONFIG['background_color']), 'black'
        
        # Select background type based on distribution
        bg_types = list(CONFIG['background_distribution'].keys())
        bg_weights = list(CONFIG['background_distribution'].values())
        
        valid_types = [(bg_type, weight) for bg_type, weight in zip(bg_types, bg_weights) if weight > 0]
        if not valid_types:
            return Image.new("RGB", (CONFIG['canvas_width'], CONFIG['canvas_height']), CONFIG['background_color']), 'black'
        
        bg_types, bg_weights = zip(*valid_types)
        selected_bg = random.choices(bg_types, weights=bg_weights)[0]
        
        if selected_bg == 'black':
            return Image.new("RGB", (CONFIG['canvas_width'], CONFIG['canvas_height']), CONFIG['background_color']), 'black'
        elif selected_bg == 'white_noise':
            return self.create_white_noise_background(), 'white_noise'
        elif selected_bg == 'plastic_texture':
            return self.create_plastic_texture_background(), 'plastic_texture'
        else:
            return Image.new("RGB", (CONFIG['canvas_width'], CONFIG['canvas_height']), CONFIG['background_color']), 'black'
    
    # =============================================================================
    # EGG PLACEMENT
    # =============================================================================
    
    def find_stick_area_in_final_canvas(self, canvas, position_data):
        """Find stick area using more precise brown/wood color detection"""
        canvas_array = np.array(canvas)
        
        # More precise stick detection - look for brown/wood colors
        stick_mask = (
            (canvas_array[:, :, 0] >= 50) &   # R >= 50
            (canvas_array[:, :, 1] >= 30) &   # G >= 30  
            (canvas_array[:, :, 2] >= 20) &   # B >= 20
            (canvas_array[:, :, 0] <= 200) &  # R <= 200
            (canvas_array[:, :, 1] <= 150) &  # G <= 150
            (canvas_array[:, :, 2] <= 120) &  # B <= 120
            # Additional check: stick colors should have R > G > B (brownish)
            (canvas_array[:, :, 0] > canvas_array[:, :, 1]) &
            (canvas_array[:, :, 1] >= canvas_array[:, :, 2])
        )
        
        if np.any(stick_mask):
            rows = np.any(stick_mask, axis=1)
            cols = np.any(stick_mask, axis=0)
            
            if np.any(rows) and np.any(cols):
                y_min, y_max = np.where(rows)[0][[0, -1]]
                x_min, x_max = np.where(cols)[0][[0, -1]]
                
                # Add safety margin
                margin = 10
                x_min = max(0, x_min + margin)
                y_min = max(0, y_min + margin)
                x_max = min(CONFIG['canvas_width'] - 1, x_max - margin)
                y_max = min(CONFIG['canvas_height'] - 1, y_max - margin)
                
                return {
                    'x_min': int(x_min),
                    'y_min': int(y_min),
                    'x_max': int(x_max),
                    'y_max': int(y_max)
                }
        
        return None

    def add_simple_scratches(self, stick):
        """Add realistic scratch effects"""
        if not CONFIG['apply_scratches']:
            return stick
        
        scratch_layer = Image.new("RGBA", stick.size, (0, 0, 0, 0))
        draw = ImageDraw.Draw(scratch_layer)
        
        mask = self.create_stick_mask()
        mask_pixels = mask.load()
        
        num_scratches = random.randint(*CONFIG['scratch_count_range'])
        
        for _ in range(num_scratches):
            # Find valid starting position
            attempts = 0
            while attempts < 50:
                x1 = random.randint(10, stick.size[0] - 10)
                y1 = random.randint(10, stick.size[1] - 10)
                if mask_pixels[x1, y1] > 0:
                    break
                attempts += 1
            
            if attempts >= 50:
                continue
            
            # Create scratch
            length = random.randint(*CONFIG['scratch_length_range'])
            angle = random.uniform(-math.pi/12, math.pi/12)  # Mostly vertical
            
            x2 = x1 + int(length * math.sin(angle))
            y2 = y1 + int(length * math.cos(angle))
            
            x2 = max(0, min(stick.size[0] - 1, x2))
            y2 = max(0, min(stick.size[1] - 1, y2))
            
            base_alpha = random.randint(*CONFIG['scratch_alpha_range'])
            width = random.randint(*CONFIG['scratch_width_range'])
            
            scratch_colors = [(255, 245, 230), (240, 220, 200), (250, 240, 220), (230, 210, 190)]
            base_color = random.choice(scratch_colors)
            
            self.draw_faded_line(draw, (x1, y1), (x2, y2), base_color, base_alpha, width)
        
        return Image.alpha_composite(stick, scratch_layer)
    
    def draw_faded_line(self, draw, start, end, color, base_alpha, width):
        """Draw a line with fade effect"""
        x1, y1 = start
        x2, y2 = end
        
        dx = x2 - x1
        dy = y2 - y1
        length = max(abs(dx), abs(dy))
        
        if length == 0:
            return
        
        segments = min(length, 50)
        
        for i in range(segments):
            t = i / max(1, segments - 1)
            
            curr_x = int(x1 + dx * t)
            curr_y = int(y1 + dy * t)
            next_x = int(x1 + dx * (t + 1/segments)) if i < segments - 1 else x2
            next_y = int(y1 + dy * (t + 1/segments)) if i < segments - 1 else y2
            
            fade_factor = 1.0 - 0.7 * (2 * abs(t - 0.5)) ** 2
            fade_factor = max(0.3, fade_factor)
            
            segment_alpha = int(base_alpha * fade_factor)
            segment_color = (*color, segment_alpha)
            
            draw.line([(curr_x, curr_y), (next_x, next_y)], fill=segment_color, width=width)

    def is_position_on_stick(self, canvas, x, y, position_data):
        """Check if position is on stick using alpha channel of rotated stick"""
        try:
            # Convert canvas coordinates to rotated stick coordinates
            stick_x = x - position_data['final_x']
            stick_y = y - position_data['final_y']
            
            # Check if we have the rotated stick and coordinates are within bounds
            if (hasattr(self, 'last_rotated_stick') and 
                0 <= stick_x < self.last_rotated_stick.width and 
                0 <= stick_y < self.last_rotated_stick.height):
                
                # Check alpha channel of rotated stick
                if self.last_rotated_stick.mode == 'RGBA':
                    pixel = self.last_rotated_stick.getpixel((stick_x, stick_y))
                    return len(pixel) >= 4 and pixel[3] > 128  # Alpha > 128 = stick area
            
            # Fallback to color detection if alpha check fails
            pixel = canvas.getpixel((x, y))
            if isinstance(pixel, tuple) and len(pixel) >= 3:
                r, g, b = pixel[0], pixel[1], pixel[2]
                return r > 30 or g > 20 or b > 10
            
            return False
        except:
            return False
    
    def place_eggs_on_final_canvas(self, canvas, stick_area_info, position_data):
        """Place eggs directly on the final canvas within the stick area with clustering"""
        
        if stick_area_info is None:
            return canvas, []
        
        # Generate random number of eggs using organic distribution
        if random.random() < 0.2:  # 20% chance of no eggs
            return canvas, []
        
        num_eggs = generate_organic_egg_count()  # Use organic distribution
        print(f" Target: {num_eggs} eggs")  # Debug line for number of eggs
        
        if num_eggs == 0:
            return canvas, []
        
        egg_annotations = []
        placed_eggs = 0
        max_total_attempts = CONFIG['placement_attempts'] * num_eggs
        attempts = 0
        
        # Calculate stick dimensions for clustering
        stick_width = stick_area_info['x_max'] - stick_area_info['x_min']
        stick_height = stick_area_info['y_max'] - stick_area_info['y_min']
        
        # Ensure we have minimum dimensions
        if stick_height < 20 or stick_width < 20:
            print(f" Stick area too small ({stick_width}x{stick_height}), skipping eggs")
            return canvas, []
        
        # Calculate center area bounds with safety checks
        center_area_ratio = CONFIG.get('center_area_ratio', 0.5)
        center_bias = CONFIG.get('center_bias', 0.85)
        
        center_margin = int(stick_height * (1 - center_area_ratio) / 2)
        center_y_min = stick_area_info['y_min'] + center_margin
        center_y_max = stick_area_info['y_max'] - center_margin
        
        # Ensure center area is valid
        if center_y_max <= center_y_min:
            center_y_min = stick_area_info['y_min'] + 5
            center_y_max = stick_area_info['y_max'] - 5
        
        margin = CONFIG['edge_margin']
        effective_margin = min(margin, min(stick_width // 4, stick_height // 4, 20))
        
        # Decide if we should create clusters
        use_clusters = random.random() < CONFIG['cluster_probability']
        cluster_centers = []
        
        if use_clusters and num_eggs > 8:
            num_clusters = random.randint(1, CONFIG['max_clusters_per_stick'])
            
            # Adjust cluster radius based on stick size
            max_cluster_radius = min(CONFIG['cluster_radius'], 
                                   stick_width // 4, 
                                   stick_height // 6)
            cluster_radius = max(10, max_cluster_radius)  # Minimum radius of 10
            
            # Create cluster centers (prefer center area)
            for cluster_idx in range(num_clusters):
                for _ in range(50):
                    # 80% chance to place cluster in center area
                    if random.random() < center_bias:
                        # Center area placement
                        x_min = stick_area_info['x_min'] + effective_margin + cluster_radius
                        x_max = stick_area_info['x_max'] - effective_margin - cluster_radius
                        y_min = center_y_min + cluster_radius
                        y_max = center_y_max - cluster_radius
                        
                        # Ensure valid ranges
                        if x_max <= x_min:
                            x_max = x_min + 1
                        if y_max <= y_min:
                            y_max = y_min + 1
                            
                        center_x = random.randint(x_min, x_max)
                        center_y = random.randint(y_min, y_max)
                    else:
                        # Outer area placement
                        x_min = stick_area_info['x_min'] + effective_margin + cluster_radius
                        x_max = stick_area_info['x_max'] - effective_margin - cluster_radius
                        y_min = stick_area_info['y_min'] + effective_margin + cluster_radius
                        y_max = stick_area_info['y_max'] - effective_margin - cluster_radius
                        
                        # Ensure valid ranges
                        if x_max <= x_min:
                            x_max = x_min + 1
                        if y_max <= y_min:
                            y_max = y_min + 1
                            
                        center_x = random.randint(x_min, x_max)
                        center_y = random.randint(y_min, y_max)
                    
                    if self.is_position_on_stick(canvas, center_x, center_y, position_data):
                        cluster_centers.append((center_x, center_y))
                        break
        
        # Place clustered eggs first
        for cluster_idx, cluster_center in enumerate(cluster_centers):
            cluster_eggs = random.randint(CONFIG['eggs_per_cluster_min'], CONFIG['eggs_per_cluster_max'])
            cluster_eggs = min(cluster_eggs, num_eggs - placed_eggs)
            
            for _ in range(cluster_eggs):
                if placed_eggs >= num_eggs:
                    break
                    
                # Place egg near cluster center with tighter distribution
                angle = random.uniform(0, 2 * math.pi)
                # Use smaller radius for tighter clustering
                distance = random.uniform(0, cluster_radius * 0.8)
                
                x = int(cluster_center[0] + distance * math.cos(angle))
                y = int(cluster_center[1] + distance * math.sin(angle))
                
                # Check bounds and stick area
                if (effective_margin <= x < CONFIG['canvas_width'] - effective_margin and 
                    effective_margin <= y < CONFIG['canvas_height'] - effective_margin):
                    
                    if self.is_position_on_stick(canvas, x, y, position_data):
                        egg_result = self.egg_generator.create_realistic_egg(
                            canvas, x, y, 
                            allow_overlap=random.random() < CONFIG['cluster_overlap_chance']
                        )
                        if egg_result:
                            canvas, yolo_annotation = egg_result
                            egg_annotations.append(yolo_annotation)
                            placed_eggs += 1
        
        # Place remaining eggs with center bias
        center_eggs_target = int((num_eggs - placed_eggs) * center_bias)
        center_eggs_placed = 0

        while placed_eggs < num_eggs and attempts < max_total_attempts:
            attempts += 1
            
            # Determine if this egg should go in center or outer area
            place_in_center = center_eggs_placed < center_eggs_target
            
            if place_in_center:
                # Center area placement - ensure valid ranges
                x_min = stick_area_info['x_min'] + effective_margin
                x_max = stick_area_info['x_max'] - effective_margin
                y_min = center_y_min + effective_margin
                y_max = center_y_max - effective_margin
                
                # Ensure valid ranges
                if x_max <= x_min:
                    x_max = x_min + 1
                if y_max <= y_min:
                    y_max = y_min + 1
                
                x = random.randint(x_min, x_max)
                y = random.randint(y_min, y_max)
            else:
                # Outer area placement - ensure valid ranges
                x_min = stick_area_info['x_min'] + effective_margin
                x_max = stick_area_info['x_max'] - effective_margin
                y_min = stick_area_info['y_min'] + effective_margin
                y_max = stick_area_info['y_max'] - effective_margin
                
                # Ensure valid ranges
                if x_max <= x_min:
                    x_max = x_min + 1
                if y_max <= y_min:
                    y_max = y_min + 1
                
                x = random.randint(x_min, x_max)
                y = random.randint(y_min, y_max)
            
            if self.is_position_on_stick(canvas, x, y, position_data):
                egg_result = self.egg_generator.create_realistic_egg(canvas, x, y, allow_overlap=False)
                if egg_result:
                    canvas, yolo_annotation = egg_result
                    egg_annotations.append(yolo_annotation)
                    placed_eggs += 1
                    
                    if place_in_center:
                        center_eggs_placed += 1
        
        print(f" Placed: {placed_eggs} eggs (clusters: {len(cluster_centers)})") #print amount of eggs and clusters
        return canvas, egg_annotations
    
    def position_and_rotate_stick(self, stick):
        """Position and rotate stick on canvas"""
        # Random rotation
        rotation_angle = random.uniform(-CONFIG['max_rotation_degrees'], CONFIG['max_rotation_degrees'])
        rotated_stick = stick.rotate(rotation_angle, expand=True, fillcolor=(0, 0, 0, 0))
        
        # Store the rotated stick for later alpha checking
        self.last_rotated_stick = rotated_stick
        
        # Center with small random offset
        base_x = (CONFIG['canvas_width'] - rotated_stick.size[0]) // 2
        base_y = (CONFIG['canvas_height'] - rotated_stick.size[1]) // 2
        
        offset_x = random.randint(-CONFIG['positioning_margin'], CONFIG['positioning_margin'])
        offset_y = random.randint(-CONFIG['positioning_margin'], CONFIG['positioning_margin'])
        
        final_x = base_x + offset_x
        final_y = base_y + offset_y
        
        # Create background
        canvas, background_type = self.create_background()
        
        # Paste stick with alpha blending
        canvas.paste(rotated_stick, (final_x, final_y), rotated_stick)
        
        return canvas, {
            'rotation_angle': rotation_angle, 
            'final_x': final_x, 
            'final_y': final_y,
            'background_type': background_type,
            'rotated_stick_size': rotated_stick.size
        }
    
    # =============================================================================
    # MAIN GENERATION PIPELINE
    # =============================================================================
    
    def generate_stick_with_eggs(self, stick_index):
        """Generate complete stick with labels and eggs"""
        try:
            stick, texture_selection = self.create_base_stick(stick_index)
            
            # LAYER 1: Add scratches FIRST (before labels and overlays)
            if CONFIG['apply_scratches']:
                stick = self.add_simple_scratches(stick)
            
            # LAYER 2: Generate and add label
            label_data = self.generate_random_label_format()
            
            # Calculate label dimensions
            label_height = int(CONFIG['actual_stick_height'] * CONFIG['label_area_height_ratio'] * CONFIG['label_height_usage'])
            label_width = int(CONFIG['actual_stick_width'] * CONFIG['label_width_ratio'])
            
            # Get font
            font_path = self.fonts[stick_index % len(self.fonts)] if self.fonts else None
            font_name = os.path.basename(font_path) if font_path else "default"
            
            # Create label
            label_img = self.create_multi_line_label(label_data, (label_width, label_height), font_path)
            
            # Position label
            label_x_offset = (CONFIG['actual_stick_width'] - label_width) // 2
            label_y_offset = int(CONFIG['actual_stick_height'] * CONFIG['label_area_height_ratio'] * CONFIG['label_y_position'])
            
            # Paste label
            stick.paste(label_img, (label_x_offset, label_y_offset), label_img)
            
            # LAYER 3: Apply overlays AFTER labels
            try:
                stick, overlay_applied, overlay_info = self.apply_overlay_texture(stick, stick_index)
                texture_selection['overlay_applied'] = overlay_applied
                texture_selection['overlay_info'] = overlay_info
            except Exception as e:
                print(f" Error in overlay application: {e}")
                texture_selection['overlay_applied'] = False
                texture_selection['overlay_info'] = "none"
            
            # STEP 1: Position and rotate the stick (WITHOUT eggs) on canvas FIRST
            canvas, position_data = self.position_and_rotate_stick(stick)
            
            # STEP 2: Find where the stick actually is in the final canvas
            stick_area_info = self.find_stick_area_in_final_canvas(canvas, position_data)
            
            # STEP 3: Place eggs on the final positioned canvas
            final_canvas, egg_annotations = self.place_eggs_on_final_canvas(canvas, stick_area_info, position_data)
            
            return final_canvas, egg_annotations, texture_selection
            
        except Exception as e:
            print(f" Error generating stick {stick_index}: {e}")
            return None, None, None

#===========================================================================================================================
# SAVING DATASET 
#===========================================================================================================================

    def determine_split(self, stick_index, total_sticks):
        """Determine which split this stick should go to"""
        # Calculate split boundaries
        train_count = int(total_sticks * CONFIG['train_ratio'])
        val_count = int(total_sticks * CONFIG['val_ratio'])
        
        if stick_index < train_count:
            return 'train'
        elif stick_index < train_count + val_count:
            return 'val'
        else:
            return 'test'
    
    def save_stick_and_annotations(self, image, annotations, stick_index, split):
        """Save stick image and YOLO annotations to appropriate directories"""
        
        # Determine output directories
        if split == 'train':
            img_dir = self.train_images_dir
            label_dir = self.train_labels_dir
        elif split == 'val':
            img_dir = self.val_images_dir
            label_dir = self.val_labels_dir
        else:  # test
            img_dir = self.test_images_dir
            label_dir = self.test_labels_dir
        
        # Save image
        img_filename = f"stick_{stick_index:04d}.jpg"
        img_path = img_dir / img_filename
        
        # Convert RGBA to RGB before saving as JPEG
        if image.mode == 'RGBA':
            image = image.convert('RGB')
        
        image.save(img_path, quality=95)
        
        # Save annotations
        label_filename = f"stick_{stick_index:04d}.txt"
        label_path = label_dir / label_filename
        
        with open(label_path, 'w') as f:
            for annotation in annotations:
                f.write(annotation + '\n')
        
        return {
            'image_path': str(img_path),
            'label_path': str(label_path),
            'split': split,
            'num_eggs': len(annotations)
        }
    
    def generate_all_sticks_with_eggs(self):
        """Generate all sticks with eggs and save in YOLO format"""
        
        all_metadata = []
        generated = 0
        total_eggs = 0
        split_counts = {'train': 0, 'val': 0, 'test': 0}
        split_eggs = {'train': 0, 'val': 0, 'test': 0}
        
        for i in range(CONFIG['num_sticks']):
            try:
                # Determine split for this stick
                split = self.determine_split(i, CONFIG['num_sticks'])
                
                # Generate stick with eggs
                stick_image, egg_annotations, texture_selection = self.generate_stick_with_eggs(i)
                
                if stick_image is None:
                    print(f" Skipping stick {i} due to generation error")
                    continue
                
                # Save image and annotations
                saved_data = self.save_stick_and_annotations(stick_image, egg_annotations, i, split)
                
                all_metadata.append(saved_data)
                
                # Update counters
                generated += 1
                num_eggs = len(egg_annotations)
                total_eggs += num_eggs
                split_counts[split] += 1
                split_eggs[split] += num_eggs
        
        # Save summary and create YOLO config
        self.save_generation_summary(all_metadata, generated, total_eggs, split_counts, split_eggs)
        self.create_yolo_config()
        
        # Print final summary
        print(f"\n GENERATION COMPLETE!")
        print(f" Generated: {generated}/{CONFIG['num_sticks']} sticks with {total_eggs} total eggs")
        
        if generated > 0:
            print(f" Average eggs per stick: {total_eggs/generated:.1f}")
        
        print(f"\n DATASET SPLIT:")
        for split in ['train', 'val', 'test']:
            sticks = split_counts[split]
            eggs = split_eggs[split]
            stick_pct = (sticks / generated * 100) if generated > 0 else 0
            print(f"  • {split.capitalize()}: {sticks} sticks ({stick_pct:.1f}%) - {eggs} eggs")
        
        return self.output_dir, generated
    
    def save_generation_summary(self, all_metadata, generated, total_eggs, split_counts, split_eggs):
        """Save generation summary to JSON"""
        summary = {
            'config': CONFIG,
            'total_sticks_generated': generated,
            'total_eggs_placed': total_eggs,
            'average_eggs_per_stick': total_eggs / generated if generated > 0 else 0,
            'split_distribution': {
                'train': {'sticks': split_counts['train'], 'eggs': split_eggs['train']},
                'val': {'sticks': split_counts['val'], 'eggs': split_eggs['val']},
                'test': {'sticks': split_counts['test'], 'eggs': split_eggs['test']}
            },
            'sticks': all_metadata
        }
        
        summary_path = self.output_dir / 'generation_summary.json'
        with open(summary_path, 'w') as f:
            json.dump(summary, f, indent=2)
        
        print(f" Generation summary saved to: {summary_path}")
    
    def create_yolo_config(self):
        """Create dataset.yaml for YOLO training"""
        config_content = f"""# YOLO dataset configuration for mosquito egg detection
path: {self.output_dir.absolute()}
train: images/train
val: images/val
test: images/test

# Classes
nc: 1  # number of classes
names: ['mosquito_egg']  # class names
"""
        
        config_path = self.output_dir / 'dataset.yaml'
        with open(config_path, 'w') as f:
            f.write(config_content)
        
        print(f" YOLO config saved to: {config_path}")

# =============================================================================
# MAIN EXECUTION
# =============================================================================

def main():
    """Generate sticks with mosquito eggs in YOLO format"""
    # Initialize generator
    generator = CombinedStickEggGenerator()
    # Generate all sticks with eggs
    output_dir, generated = generator.generate_all_sticks_with_eggs()
    
    print(f"\n GENERATION COMPLETE!")
    print(f" Dataset ready at: {output_dir}")

if __name__ == "__main__":
    main()